In [ ]:
import os
import torch
import numpy as np
import tensorflow as tf
import dataloader as dset
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Multiply
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D

In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

LMDB_PATH_HOST="..\dataset\lmdb.hwr_40-1.0"
TRN_DATA="..\dataset\data\lines.filtered_max_width.tst.55.shuf"
TST_DATA="..\dataset\datalines.filtered_max_width.tst.55"

In [ ]:
def create_pairs(images, labels):
  imagePairs = []
  labelPairs = []

  numclasses = len(np.unique(labels))
  idx = [np.where(labels ==i)[0] for i in range(numclasses)]

  for ind in range(len(images)):
    currImage = images[ind]
    label = labels[ind]

    indB = np.random.choice(idx[label])
    indImage = images[indB]

    imagePairs.append([currImage, indImage])

    labelPairs.append([1])

    diss_idx = np.where(labels != label)[0]

    diss_image = images[np.random.choice(diss_idx)]

    imagePairs.append([currImage, diss_image])
    labelPairs.append([0])

  return (np.array(imagePairs), np.array(labelPairs))

In [ ]:
train_set = dset.DatasetFromLMDB(lmdb_path=LMDB_PATH_HOST, labels_path=TRN_DATA)

X_train = []
y_train = []

for index in range(len(train_set)):
    image, label, image_name = train_set[index]
    X_train.append(image)
    y_train.append(label.item())

X_tensor = torch.stack(X_train)
X_train = X_tensor.numpy()
y_train = np.array(y_train)
(training_pairs, training_labels) = create_pairs(X_train, y_train)

In [ ]:
from sklearn.model_selection import train_test_split
train_pairs, test_pairs, train_labels, test_labels = train_test_split(training_pairs, training_labels, test_size=0.2, random_state=42)

training_pairs = np.transpose(training_pairs, (0, 1, 3, 4, 2))
test_pairs = np.transpose(test_pairs, (0, 1, 3, 4, 2))

In [ ]:
def euclidean_distance(vecs):
  (imgA, imgB) = vecs
  ss = K.sum(K.square(imgA - imgB), axis = 1, keepdims=True)
  return K.sqrt(K.maximum(ss, K.epsilon()))

def contrastiveLoss(y, y_preds, margin=1):
    y = tf.cast(y, y_preds.dtype)
    y_preds_squared = K.square(y_preds)
    margin_squared = K.square(K.maximum(margin - y_preds, 0))
    loss = K.mean(y * y_preds_squared + (1 - y) * margin_squared)
    return loss
    
def siamese_model(input_shape, embeddingDim=48, use_pretrained=True):
    inputs = Input(input_shape)
    if use_pretrained:
        base_model = VGG16(include_top=False, input_shape=(input_shape[0], input_shape[1], 3))
        x = base_model(inputs)
    else:
        x = inputs

    # Prvá konvolučná vrstva s pozornosťou
    conv1 = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    attention1 = Conv2D(64, (3, 3), activation='sigmoid', padding='same')(x)
    x = Multiply()([conv1, attention1])
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)  # pridané padding='same'
    x = Dropout(0.3)(x)

    # Druhá konvolučná vrstva s pozornosťou
    conv2 = Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    attention2 = Conv2D(128, (3, 3), activation='sigmoid', padding='same')(x)
    x = Multiply()([conv2, attention2])
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)  # pridané padding='same'
    x = Dropout(0.3)(x)

    # Tretia konvolučná vrstva a GAP
    x = Conv2D(256, (3, 3), activation="relu", padding="same")(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)

    # Fully connected layer pre embedding
    outputs = Dense(embeddingDim)(x)

    model = Model(inputs, outputs)
    return model


In [ ]:
image_shape = (200, 50, 3)
batch_size = 64
epochs = 15

imageA = Input(shape = image_shape)
imageB = Input(shape = image_shape)

model_build = siamese_model(image_shape)
modelA = model_build(imageA)
modelB = model_build(imageB)

distance = Lambda(euclidean_distance)([modelA, modelB])
model = Model(inputs=[imageA, imageB], outputs=distance)

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.compile(loss = contrastiveLoss, optimizer="adam")

history = model.fit(
    [training_pairs[:, 0], training_pairs[:, 1]], training_labels[:],
    validation_data=([test_pairs[:, 0], test_pairs[:, 1]], test_labels[:]),
    batch_size = batch_size,
    epochs = epochs,
    callbacks=[early_stopping_callback])